In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np


In [38]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

Data Exploration

In [4]:
train.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,NaN,-5.619088,0,0.082570,158.386236,4,0.734642,0
1,1,NaN,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1,0.127358,102.752988,3,0.711531,1
2,2,193213.0,NaN,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0,0.052282,178.685791,3,0.425536,0
3,3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0,0.035618,128.715630,3,0.453597,0
4,4,165969.0,0.493017,NaN,0.740982,0.002033,10.0,0.094891,-2.684095,0,0.050746,121.928157,4,0.741311,0


In [5]:
test.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,0,308523.0,0.019845,NaN,0.908939,0.001438,NaN,0.112832,-8.890172,0,0.082714,126.129304,4,0.399620
1,1,200011.0,0.070119,0.731256,0.444655,0.002020,10.0,0.139040,-6.301214,0,0.061685,86.448149,3,0.499424
2,2,279758.0,0.810637,0.568858,0.125466,0.898841,0.0,0.226614,-11.542478,0,0.041868,99.544351,3,0.564951
3,3,249197.0,NaN,0.871789,0.557342,0.000715,4.0,0.325391,-7.905546,1,0.046815,123.063854,4,0.906485
4,4,NaN,0.765568,0.624687,0.710794,0.000346,8.0,0.308284,NaN,0,0.129284,88.703121,3,0.935571


In [13]:
train.isnull().sum()/train.shape[0] * 100

id                   0.0000
song_duration_ms    10.2525
acousticness         9.9800
danceability        10.0650
energy               9.9375
instrumentalness     9.9625
key                 10.1625
liveness            10.2150
loudness             9.8925
audio_mode           0.0000
speechiness          0.0000
tempo                0.0000
time_signature       0.0000
audio_valence        0.0000
song_popularity      0.0000
dtype: float64

This shows train and test data have null values and checking the percentages of null values it resulted a very less percentage and no need of removing those columns but filling them with imputation process.

In [14]:
null_col = ['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness']

In [30]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()

In [45]:
X_train = train.drop(['id', 'song_popularity'], axis=1)
y_train = train['song_popularity']

In [46]:
train_impute = imputer.fit_transform(X_train.copy())

In [62]:
train_iterative_impute = pd.DataFrame(train_impute, columns=list(X_train.columns))

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [66]:
logreg=LogisticRegression()
kf=KFold(n_splits=5)
score=cross_val_score(logreg,train_iterative_impute,y_train,cv=kf)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores are [0.63475  0.631625 0.637875 0.638875 0.634875]
Average Cross Validation score :0.6355999999999999


cvv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
rf = RandomForestClassifier()
score=cross_val_score(rf,train_simple_impute,y_train,cv=cvv)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

In [59]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy ='mean')

In [60]:
data = imputer.fit_transform(X_train.copy())

In [65]:
train_simple_impute = pd.DataFrame(data, columns=list(X_train.columns))

In [70]:
from sklearn.ensemble import RandomForestRegressor
#regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

In [72]:
logreg=LogisticRegression()
kf=KFold(n_splits=5)
score=cross_val_score(logreg,train_simple_impute,y_train,cv=kf)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores are [0.63475  0.631625 0.637875 0.638875 0.634875]
Average Cross Validation score :0.6355999999999999


so by cross validation score its came to know that simple impute and iterative impute are performing the same on handling missing values.


In [78]:
test_data = imputer.transform(test.copy())
test_simple_impute = pd.DataFrame(test_data, columns=list(test.columns))

In [79]:
test_simple_impute.isnull().sum()

id                  0
song_duration_ms    0
acousticness        0
danceability        0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
audio_mode          0
speechiness         0
tempo               0
time_signature      0
audio_valence       0
dtype: int64

In [80]:
X = train_simple_impute
Y = y_train 

In [81]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [82]:
logreg.fit(x_train, y_train)

LogisticRegression()

In [83]:
predictions = logreg.predict(x_test)


In [84]:
score = logreg.score(x_test, y_test)
print(score)

0.6377


In [98]:
dataset = imputer.fit_transform(train.copy())

In [99]:
train_simple_imputer= pd.DataFrame(dataset, columns=list(train.columns))

In [100]:
train_simple_imputer.head()

,id,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,song_popularity
0,0.0,212990.000000,0.642286,0.856520,0.707073,0.002001,10.0,0.198514,-5.619088,0.0,0.082570,158.386236,4.0,0.734642,0.0
1,1.0,193165.847572,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1.0,0.127358,102.752988,3.0,0.711531,1.0
2,2.0,193213.000000,0.276404,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0.0,0.052282,178.685791,3.0,0.425536,0.0
3,3.0,249893.000000,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0.0,0.035618,128.715630,3.0,0.453597,0.0
4,4.0,165969.000000,0.493017,0.570951,0.740982,0.002033,10.0,0.094891,-2.684095,0.0,0.050746,121.928157,4.0,0.741311,0.0


In [127]:
import scipy

In [129]:
column = ['liveness', 'speechiness']
for col in column:
    train_simple_imputer[col], filterd_lmbda = scipy.stats.boxcox(train_simple_imputer[col], lmbda=None)

In [130]:
X = train_simple_imputer.drop(['id', 'song_popularity'], axis=1)
Y = train_simple_imputer['song_popularity']

In [102]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier

In [103]:
log_reg = LogisticRegression()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()

#xgboost = XGBClassifier()
#lgbm = LGBMClassifier()
#cat = CatBoostClassifier()

In [107]:
from sklearn.metrics import roc_auc_score

In [133]:
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=1)

In [134]:
models = {"Logistice regression": log_reg,"Decision tree": decision_tree, "Random Forest": random_forest}

for model_name, model in models.items():
    print(model_name)
    model.fit(X_train, y_train)
    y_pred = model.predict(x_test)
    auc = roc_auc_score(y_test, y_pred)
    print("AUC of the {} ---> {}".format(model_name, auc))

Logistice regression
AUC of the Logistice regression ---> 0.5
Decision tree
AUC of the Decision tree ---> 0.5074703209055318
Random Forest
AUC of the Random Forest ---> 0.5100933549001764


In [117]:
test_data = test_simple_impute.drop(['id'], axis=1)

In [125]:
y_pred = random_forest.predict(test_data)
y_pred = y_pred.astype(int)
test['song_popularity'] = y_pred
submission = test[['id', 'song_popularity']]
submission

,id,song_popularity
0,0,1
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
9995,9995,0
9996,9996,0
9997,9997,0
9998,9998,1


In [126]:
submission.to_csv('submission.csv', index=False)

In [135]:
t_liveness_log = np.log(test['liveness'])
t_liveness_log.skew()

0.9503113380758493

In [137]:
test['liveness'] = t_liveness_log


In [138]:
train.skew()

id                  0.000000
song_duration_ms    0.632932
acousticness        1.004194
danceability       -0.398386
energy             -0.582832
instrumentalness    4.947631
key                -0.075304
liveness            2.165546
loudness           -1.430100
audio_mode          0.766116
speechiness         1.720121
tempo               0.764150
time_signature      0.254061
audio_valence      -0.181006
song_popularity     0.563540
dtype: float64

In [139]:
test.skew()

id                  0.000000
song_duration_ms    0.624242
acousticness        0.984593
danceability       -0.398803
energy             -0.572329
instrumentalness    4.949335
key                -0.063806
liveness            0.950311
loudness           -1.410155
audio_mode          0.753226
speechiness         1.695977
tempo               0.798142
time_signature      0.220661
audio_valence      -0.145573
song_popularity     3.647383
dtype: float64

In [140]:
t_instrumentalness_log = np.log(test['instrumentalness'])
t_instrumentalness_log.skew()

c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


1.5051342456009544

In [141]:
t_instrumentalness_log

0      -6.544156
1      -6.204810
2      -0.106649
3      -7.243249
4      -7.967652
          ...   
9995   -5.812630
9996   -6.260317
9997   -2.009715
9998   -9.929260
9999   -3.277392
Name: instrumentalness, Length: 10000, dtype: float64

In [142]:
test['instrumentalness'] = t_instrumentalness_log

In [143]:
train['instrumentalness'] = np.log(train['instrumentalness'])
train['liveness'] = np.log(train['liveness']) 

In [144]:
train['liveness'] = np.log(train['liveness']) 

In [148]:
train.skew().sort_values(ascending = False)

speechiness         1.720121
instrumentalness    1.544011
acousticness        1.004194
liveness            0.942534
audio_mode          0.766116
tempo               0.764150
song_duration_ms    0.632932
song_popularity     0.563540
time_signature      0.254061
id                  0.000000
key                -0.075304
audio_valence      -0.181006
danceability       -0.398386
energy             -0.582832
loudness           -1.430100
dtype: float64

In [149]:
train_simple_imputer['instrumentalness'] = np.log(train_simple_imputer['instrumentalness'])
train_simple_imputer['liveness'] = np.log(train_simple_imputer['liveness']) 

In [150]:
test_simple_impute['instrumentalness'] = np.log(test_simple_impute['instrumentalness'])
test_simple_impute['liveness'] = np.log(test_simple_impute['liveness']) 

In [164]:
X = train_simple_imputer.drop(['id', 'song_popularity'], axis=1)
Y = train_simple_imputer['song_popularity']

In [169]:
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=1)

In [170]:
models = {"Logistice regression": log_reg,"Decision tree": decision_tree, "Random Forest": random_forest}

for model_name, model in models.items():
    print(model_name)
    model.fit(X_train, y_train)
    y_pred = model.predict(x_test)
    auc = roc_auc_score(y_test, y_pred)
    print("AUC of the {} ---> {}".format(model_name, auc))

Logistice regression
AUC of the Logistice regression ---> 0.5
Decision tree
AUC of the Decision tree ---> 0.5047246823924241
Random Forest
AUC of the Random Forest ---> 0.5139425634987568


In [156]:
dataset = imputer.fit_transform(train_simple_imputer.copy())
train_simple_imputer= pd.DataFrame(dataset, columns=list(train_simple_imputer.columns))

In [159]:
test_simple_impute.head().shape


(5, 14)

In [160]:
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()

In [162]:
dataset_test = imputer.fit_transform(test_simple_impute.copy())

In [163]:
test_simple_impute= pd.DataFrame(dataset_test, columns=list(test_simple_impute.columns))

In [172]:
from imblearn.over_sampling import SMOTE

ImportError: cannot import name '_euclidean_distances' from 'sklearn.metrics.pairwise' (c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\pairwise.py)

In [ ]:
oversample = SMOTE()
X, y = oversample.fit_resample(train_X, train_y)